This idea of this notebook is to create a first (simple) DL model using all the same features used in the multinomial logit model.

Much inspiration was derived from https://towardsdatascience.com/use-machine-learning-to-predict-horse-racing-4f1111fb6ced.

In [100]:
import math
from importlib import reload
import deeplearninglib
reload(deeplearninglib)
from deeplearninglib import *

import wandb

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [101]:
# select model to train

model_inventory = {'mktprob': {'XZ_columns': ["mkt_prob"],
                               'continuous_features': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': LinSig},
                   'mktprob_soft': {'XZ_columns': ["mkt_prob"],
                               'continuous_features': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': LinSoft},
                    'mktprob_MLR': {'XZ_columns': ["mkt_prob"],
                               'continuous_features': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': MLR},
                   'AlunOwen_v0': {'XZ_columns': ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v1': {'XZ_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v2': {'XZ_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinDropReluLinSoft},
                   'AlunOwen_v3': {'XZ_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': False,
                                   'model_architecture': MLR},
                   'AlunOwen_v4': {'XZ_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': [],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': False,
                                   'model_architecture': MLR},
                   'test_v1': {'XZ_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie", "course_Kempton", "course_Lingfield", "course_Southwell", "course_Wolverhampton", "going_Firm", "going_Good", "going_Good to Firm", "going_Good to Soft", "going_Slow", "going_Soft", "going_Standard", "going_Standard to Slow", "direction_Left Handed", "direction_Right Handed"],
                                   'continuous_features': [],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': False,
                                   'model_architecture': LinDropReluLinSoft}
                               }

model = 'test_v1'
XZ_columns = model_inventory[model]['XZ_columns']
continuous_features = model_inventory[model]['continuous_features']
learning_rate = model_inventory[model]['learning_rate']
epochs = model_inventory[model]['epochs']
vacant_stall_indicator = model_inventory[model]['vacant_stall_indicator']
bias = model_inventory[model]['bias']
model_architecture = model_inventory[model]['model_architecture']

In [102]:
# read in data

y_columns = ["win"] # assumed to be contained in runners files

runners_train_data_fn = "data\\runners_train.csv"
races_train_data_fn = "data\\races_train.csv"
runners_validate_data_fn = "data\\runners_validate.csv"
races_validate_data_fn = "data\\races_validate.csv"

train_data = RacesDataset(runners_train_data_fn, races_train_data_fn, XZ_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator, continuous_features=continuous_features)
validate_data = RacesDataset(runners_validate_data_fn, races_validate_data_fn, XZ_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator, scalar=train_data.scalar, continuous_features=continuous_features)

train_dataloader = DataLoader(train_data, batch_size=64)
validate_dataloader = DataLoader(validate_data, batch_size=64)

In [103]:
pd.options.display.max_columns = 1000 # was 20
train_data.runners_wide.loc[:, train_data.X_columns].head()

age                                                              \
stall_number   1    2    3    4    5    6    7    8    9    10   11   12   13   
race_id                                                                         
11504         3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  0.0  0.0  0.0  0.0  0.0   
11505         4.0  6.0  9.0  7.0  5.0  8.0  5.0  5.0  5.0  4.0  5.0  0.0  0.0   
11506         3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  0.0  0.0  0.0   
11507         5.0  4.0  4.0  4.0  4.0  4.0  8.0  4.0  4.0  0.0  0.0  0.0  0.0   
11508         7.0  4.0  5.0  4.0  5.0  4.0  5.0  5.0  4.0  0.0  0.0  0.0  0.0   

                            blinkers                                          \
stall_number   14   15   16       1    2    3    4    5    6    7    8    9    
race_id                                                                        
11504         0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0      1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                cheekpieces                 \
stall_number   10   11   12   13   14   15   16          1    2    3    4    
race_id                                                                      
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  1.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  1.0  1.0  0.0   

                                                                          \
stall_number   5    6    7    8    9    10   11   12   13   14   15   16   
race_id                                                                    
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             daysLTO                                                          \
stall_number      1      2      3      4      5      6      7      8      9    
race_id                                                                        
11504          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0    0.0   
11505          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11506          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11507          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11508          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   

                                                    entire                 \
stall_number     10     11   12   13   14   15   16     1    2    3    4    
race_id                                                                     
11504           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11505         365.0  365.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11506         365.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11507           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11508           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   

                                                                          \
stall_number   5    6    7    8    9    10   11   12   13   14   15   16   
race_id                                                                    
1150

In [104]:
assert model_architecture != "MLR" or train_data.races is None, "Use of MLR with race-specific variables isn't yet supported"
train_data.races.head()

,going_Standard,going_Good to Firm,course_Wolverhampton,going_Good,course_Lingfield,going_Firm,going_Slow,course_Kempton,going_Soft,direction_Left Handed,going_Standard to Slow,going_Good to Soft,direction_Right Handed,course_Southwell
race_id,,,,,,,,,,,,,,
11504,1,0,1,0,0,0,0,0,0,1,0,0,0,0
11505,1,0,1,0,0,0,0,0,0,1,0,0,0,0
11506,1,0,1,0,0,0,0,0,0,1,0,0,0,0
11507,1,0,1,0,0,0,0,0,0,1,0,0,0,0
11508,1,0,1,0,0,0,0,0,0,1,0,0,0,0


In [105]:
# build the neural network

output_layer_nodes = train_data.y.shape[1]
if train_data.Z is not None:
    input_layer_nodes = train_data.X.shape[1] + train_data.Z.shape[1]
else:
    input_layer_nodes = train_data.X.shape[1]

# torch.manual_seed(0)
net = model_architecture(input_layer_nodes, output_layer_nodes, bias=bias).to(device) # linear-relu-linear-softwax nn (1 hidden layer)
print(f"Model structure: {model}")

for name, param in net.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param} \n")

Model structure: test_v1
Layer: neural_network.0.weight | Size: torch.Size([73, 334]) | Values : Parameter containing:
tensor([[ 0.0284,  0.0407, -0.0254,  ...,  0.0311, -0.0396,  0.0508],
        [-0.0451, -0.0030,  0.0368,  ...,  0.0005,  0.0474, -0.0134],
        [ 0.0091, -0.0458,  0.0437,  ..., -0.0203,  0.0479, -0.0195],
        ...,
        [ 0.0053,  0.0403,  0.0285,  ..., -0.0418, -0.0369, -0.0417],
        [ 0.0032,  0.0400, -0.0163,  ..., -0.0114,  0.0047, -0.0427],
        [ 0.0235,  0.0023,  0.0078,  ..., -0.0309, -0.0385, -0.0161]],
       requires_grad=True) 

Layer: neural_network.3.weight | Size: torch.Size([16, 73]) | Values : Parameter containing:
tensor([[ 1.1324e-01,  2.4941e-02,  4.5150e-02,  ...,  9.1173e-02,
          7.0800e-02,  1.3934e-02],
        [-1.0043e-01, -5.3688e-03,  1.1343e-01,  ..., -1.0174e-01,
          9.3881e-02,  7.4871e-02],
        [-7.8712e-02, -1.0301e-01,  3.8820e-02,  ...,  5.2862e-03,
         -7.2134e-02, -4.2702e-02],
        ...,
   

In [106]:
# example to show how model is used from prediction

X = torch.rand(1, input_layer_nodes, device=device)
logits = net(X)
y_pred = logits.argmax(1)
print(f"Predicted class: {y_pred}")
print(X.shape)

Predicted class: tensor([11])
torch.Size([1, 334])


In [107]:
%env WANDB_NOTEBOOK_NAME 'C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="horse-racing-project",
    
    # track hyperparameters and run metadata
    config={
    "device": device,
    "model": model,
    "XZ_columns": XZ_columns,
    "continuous_featurs": continuous_features,
    "learning_rate": learning_rate,
    "epochs": epochs,
    "vacant_stall_indicator": vacant_stall_indicator,
    "bias": bias,
    "model_architecture": list(net.modules())
    }
)

env: WANDB_NOTEBOOK_NAME='C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'


In [108]:
# optimizing model parameters

# initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net, loss_fn, optimizer, device)
    (acc, loss) = validate_loop(validate_dataloader, net, loss_fn, device)
    wandb.log({"acc": acc, "loss": loss})
print("Done!")

Epoch 1
-------------------------------
loss: 2.758046  [   64/15874]
loss: 2.784117  [ 6464/15874]
loss: 2.786818  [12864/15874]
Validate Error: 
 Accuracy: 11.7%, Avg loss: 2.762081 

Epoch 2
-------------------------------
loss: 2.794769  [   64/15874]
loss: 2.794303  [ 6464/15874]
loss: 2.783925  [12864/15874]
Validate Error: 
 Accuracy: 11.2%, Avg loss: 2.764123 

Epoch 3
-------------------------------
loss: 2.793599  [   64/15874]
loss: 2.788542  [ 6464/15874]
loss: 2.766137  [12864/15874]
Validate Error: 
 Accuracy: 11.0%, Avg loss: 2.764812 

Epoch 4
-------------------------------
loss: 2.758404  [   64/15874]
loss: 2.786390  [ 6464/15874]
loss: 2.772120  [12864/15874]
Validate Error: 
 Accuracy: 11.0%, Avg loss: 2.764146 

Epoch 5
-------------------------------
loss: 2.781122  [   64/15874]
loss: 2.785617  [ 6464/15874]
loss: 2.766143  [12864/15874]
Validate Error: 
 Accuracy: 11.5%, Avg loss: 2.760255 

Epoch 6
-------------------------------
loss: 2.774837  [   64/15874]


In [109]:
# finish the wandb run, necessary in notebooks
wandb.finish()

acc,▅▂▄▅▆▅▄▄▃▇▃▃▅▃▅▄▄▆█▄▅▃▄▃▃▆▇▆▅▅▆▄▃▁▆▅▇▅▃▅
loss,▇█▇▅▄▄▄▅▄▂▄▂▃▃▃▃▃▂▁▄▂▂▃▃▂▂▂▂▃▃▃▄▃▄▂▂▃▃▅▃
acc,0.11647
loss,2.75147


In [110]:
for para_name, para_vals in net.named_parameters():
    np.savetxt(para_name + ".csv", para_vals.data.numpy(), fmt='%6.3f', delimiter=",")